In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# cargamos el dataframe correspondiente 
df = pd.read_csv("archivos/candy_production.csv")

display(df.head())

In [3]:
def exploracion(df):
    df_info = pd.DataFrame()
    df_info["% nulos"] = round(df.isna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["% no_nulos"] = round(df.notna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["tipo_dato"] = df.dtypes
    df_info["num_valores_unicos"] = df.nunique()
    print(f"""El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.
Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0], 2)}% de los datos.
Hay {len(list(df_info[(df_info["% nulos"] != "0.0%")].index))} columnas con datos nulos, y son:
{list(df_info[(df_info["% nulos"] != "0.0%")].index)}
y sin nulos hay {len(list(df_info[(df_info["% nulos"] == "0.0%")].index))} columnas y son:
{list(df_info[(df_info["% nulos"] == "0.0%")].index)}
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:""")
    display(df_info.head())
    print("Principales estadísticos de las columnas categóricas:")
    display(df.describe(include="O").T)
    print("Principales estadísticos de las columnas numéricas:")
    display(df.describe(exclude="O").T)
    return df_info

exploracion(df)

El DataFrame tiene 548 filas y 2 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 0 columnas con datos nulos, y son:
[]
y sin nulos hay 2 columnas y son:
['observation_date', 'IPG3113N']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
observation_date,0.0%,100.0%,object,548
IPG3113N,0.0%,100.0%,float64,548


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
observation_date,548,548,2017-08-01,1


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
IPG3113N,548.0,100.662524,18.052931,50.6689,87.862475,102.27855,114.6919,139.9153


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
observation_date,0.0%,100.0%,object,548
IPG3113N,0.0%,100.0%,float64,548


# Valores unicos de las variables categoricas ✨

In [4]:
# creamos una lista con los nombres de las columnas categoricas 
columnas = df.select_dtypes(include='object').columns.tolist()
print(columnas)
# empezamos a iterar por cada una de las columnas para sacar sus valores únicos y sus frecuencias
for columna in columnas:
    print(f" \n----------- ESTAMOS ANALIZANDO LA COLUMNA: '{columna.upper()}' -----------\n")
    print(f"Sus valores únicos son: {df[columna].unique()}\n")
    print(f"Las frecuencias de los valores únicos de las categorías son: {df[columna].value_counts()} ")

['observation_date']
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'OBSERVATION_DATE' -----------

Sus valores únicos son: ['1972-01-01' '1972-02-01' '1972-03-01' '1972-04-01' '1972-05-01'
 '1972-06-01' '1972-07-01' '1972-08-01' '1972-09-01' '1972-10-01'
 '1972-11-01' '1972-12-01' '1973-01-01' '1973-02-01' '1973-03-01'
 '1973-04-01' '1973-05-01' '1973-06-01' '1973-07-01' '1973-08-01'
 '1973-09-01' '1973-10-01' '1973-11-01' '1973-12-01' '1974-01-01'
 '1974-02-01' '1974-03-01' '1974-04-01' '1974-05-01' '1974-06-01'
 '1974-07-01' '1974-08-01' '1974-09-01' '1974-10-01' '1974-11-01'
 '1974-12-01' '1975-01-01' '1975-02-01' '1975-03-01' '1975-04-01'
 '1975-05-01' '1975-06-01' '1975-07-01' '1975-08-01' '1975-09-01'
 '1975-10-01' '1975-11-01' '1975-12-01' '1976-01-01' '1976-02-01'
 '1976-03-01' '1976-04-01' '1976-05-01' '1976-06-01' '1976-07-01'
 '1976-08-01' '1976-09-01' '1976-10-01' '1976-11-01' '1976-12-01'
 '1977-01-01' '1977-02-01' '1977-03-01' '1977-04-01' '1977-05-01'
 '1977-06-01' '1977-

# DUPLICADOS

In [5]:
def get_duplicate_rows(df):
    """
    Encuentra y devuelve las filas completas duplicadas de un DataFrame.

    Args:
        df (pd.DataFrame): El DataFrame de entrada.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las filas duplicadas completas.
    """
    # Identificar duplicados basados en todas las columnas
    duplicate_rows = df[df.duplicated(keep=False)]
    return duplicate_rows

get_duplicate_rows(df)

,observation_date,IPG3113N
